In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import FeatureHasher
from matplotlib import pyplot as plt
import seaborn as sns

In [2]:
!ls


data  Note1.ipynb  trials.ipynb


In [3]:
df= pd.read_csv('data/Global_EV_2023.csv')

In [4]:
df.head()

,region,category,parameter,mode,powertrain,year,unit,value
0,Australia,Historical,EV stock,Cars,BEV,2011,Vehicles,49.00000
1,Australia,Historical,EV sales share,Cars,EV,2011,percent,0.00650
2,Australia,Historical,EV stock share,Cars,EV,2011,percent,0.00046
3,Australia,Historical,EV sales,Cars,BEV,2011,Vehicles,49.00000
4,Australia,Historical,EV sales,Cars,BEV,2012,Vehicles,170.00000


In [5]:
cat_cols=[]
num_cols=[]
for cols in df.columns:
    if df[cols].dtype=="object":
        cat_cols.append(cols)
    else:
        num_cols.append(cols)

In [6]:
num_cols

['year', 'value']

In [7]:
cat_cols

['region', 'category', 'parameter', 'mode', 'powertrain', 'unit']

In [8]:
for cat in cat_cols:
    print(df[cat].isnull().sum())

0
0
0
0
0
0


In [9]:
for cat in num_cols:
    print(df[cat].isnull().sum())

0
0


In [10]:
df[cat_cols].nunique()

region        38
category       3
parameter      8
mode           5
powertrain     5
unit           6
dtype: int64

In [11]:
for cols in cat_cols:
    print(cols)
    print(df[cols].unique())
    print()

region
['Australia' 'Austria' 'Belgium' 'Brazil' 'Canada' 'Chile' 'China'
 'Denmark' 'EU27' 'Europe' 'Finland' 'France' 'Germany' 'Greece' 'Iceland'
 'India' 'Indonesia' 'Israel' 'Italy' 'Japan' 'Korea' 'Mexico'
 'Netherlands' 'New Zealand' 'Norway' 'Other Europe' 'Poland' 'Portugal'
 'Rest of the world' 'South Africa' 'Spain' 'Sweden' 'Switzerland'
 'Thailand' 'Turkiye' 'United Kingdom' 'USA' 'World']

category
['Historical' 'Projection-STEPS' 'Projection-APS']

parameter
['EV stock' 'EV sales share' 'EV stock share' 'EV sales'
 'EV charging points' 'Electricity demand' 'Oil displacement Mbd'
 'Oil displacement, million lge']

mode
['Cars' 'EV' 'Buses' 'Vans' 'Trucks']

powertrain
['BEV' 'EV' 'PHEV' 'Publicly available fast' 'Publicly available slow']

unit
['Vehicles' 'percent' 'charging points' 'GWh' 'Milion barrels per day'
 'Oil displacement, million lge']



In [12]:
for cat in num_cols:
    IQR=df[cat].quantile(0.75)-df[cat].quantile(0.25)
    UB= df[cat].quantile(0.75)+ (1.5*IQR)
    LB= df[cat].quantile(0.25)-(1.5*IQR)
    s=0
    for row in df[cat]:
        if row>=UB or row<= LB:
            s+=1
    print(f"{cat}: {s}")

year: 444
value: 1826


In [13]:
df.head()

,region,category,parameter,mode,powertrain,year,unit,value
0,Australia,Historical,EV stock,Cars,BEV,2011,Vehicles,49.00000
1,Australia,Historical,EV sales share,Cars,EV,2011,percent,0.00650
2,Australia,Historical,EV stock share,Cars,EV,2011,percent,0.00046
3,Australia,Historical,EV sales,Cars,BEV,2011,Vehicles,49.00000
4,Australia,Historical,EV sales,Cars,BEV,2012,Vehicles,170.00000


In [14]:
df1 = df[df["unit"]=="Vehicles"]

In [15]:
df2 = df[df["unit"]=="percent"]

In [16]:
df3 = df[df["unit"]=="charging points"]

In [17]:
df4 = df[df["unit"]=="GWh"]

In [18]:
df5 = df[df["unit"]=="Milion barrels per day"]

In [19]:
df6 = df[df["unit"]=="Oil displacement, million lge"]

In [20]:
df1.head()

,region,category,parameter,mode,powertrain,year,unit,value
0,Australia,Historical,EV stock,Cars,BEV,2011,Vehicles,49.0
3,Australia,Historical,EV sales,Cars,BEV,2011,Vehicles,49.0
4,Australia,Historical,EV sales,Cars,BEV,2012,Vehicles,170.0
7,Australia,Historical,EV stock,Cars,BEV,2012,Vehicles,220.0
8,Australia,Historical,EV stock,Cars,PHEV,2012,Vehicles,80.0


In [21]:
df2.head()

,region,category,parameter,mode,powertrain,year,unit,value
1,Australia,Historical,EV sales share,Cars,EV,2011,percent,0.00650
2,Australia,Historical,EV stock share,Cars,EV,2011,percent,0.00046
5,Australia,Historical,EV stock share,Cars,EV,2012,percent,0.00280
6,Australia,Historical,EV sales share,Cars,EV,2012,percent,0.03000
13,Australia,Historical,EV sales share,Cars,EV,2013,percent,0.03400


In [22]:
df3.head()

,region,category,parameter,mode,powertrain,year,unit,value
40,Australia,Historical,EV charging points,EV,Publicly available fast,2017,charging points,40.0
41,Australia,Historical,EV charging points,EV,Publicly available slow,2017,charging points,440.0
42,Australia,Historical,EV charging points,EV,Publicly available fast,2018,charging points,61.0
43,Australia,Historical,EV charging points,EV,Publicly available slow,2018,charging points,670.0
56,Australia,Historical,EV charging points,EV,Publicly available fast,2019,charging points,250.0


In [23]:
df4.head()

,region,category,parameter,mode,powertrain,year,unit,value
1002,China,Historical,Electricity demand,Buses,EV,2010,GWh,24000.0
1003,China,Historical,Electricity demand,Trucks,EV,2010,GWh,36.0
1004,China,Historical,Electricity demand,Vans,EV,2010,GWh,110.0
1005,China,Historical,Electricity demand,Cars,EV,2010,GWh,330.0
1022,China,Historical,Electricity demand,Buses,EV,2011,GWh,24000.0


In [24]:
df5.head()

,region,category,parameter,mode,powertrain,year,unit,value
1006,China,Historical,Oil displacement Mbd,Buses,EV,2010,Milion barrels per day,0.001100
1007,China,Historical,Oil displacement Mbd,Trucks,EV,2010,Milion barrels per day,0.000002
1008,China,Historical,Oil displacement Mbd,Vans,EV,2010,Milion barrels per day,0.000010
1009,China,Historical,Oil displacement Mbd,Cars,EV,2010,Milion barrels per day,0.000026
1014,China,Historical,Oil displacement Mbd,Buses,EV,2011,Milion barrels per day,0.001100


In [25]:
df6.head()

,region,category,parameter,mode,powertrain,year,unit,value
1010,China,Historical,"Oil displacement, million lge",Buses,EV,2010,"Oil displacement, million lge",68.000
1011,China,Historical,"Oil displacement, million lge",Trucks,EV,2010,"Oil displacement, million lge",0.088
1012,China,Historical,"Oil displacement, million lge",Vans,EV,2010,"Oil displacement, million lge",0.590
1013,China,Historical,"Oil displacement, million lge",Cars,EV,2010,"Oil displacement, million lge",1.500
1018,China,Historical,"Oil displacement, million lge",Buses,EV,2011,"Oil displacement, million lge",67.000


In [26]:
dn=[df1, df2,df3,df4,df5,df6]

In [27]:
df1.head()

,region,category,parameter,mode,powertrain,year,unit,value
0,Australia,Historical,EV stock,Cars,BEV,2011,Vehicles,49.0
3,Australia,Historical,EV sales,Cars,BEV,2011,Vehicles,49.0
4,Australia,Historical,EV sales,Cars,BEV,2012,Vehicles,170.0
7,Australia,Historical,EV stock,Cars,BEV,2012,Vehicles,220.0
8,Australia,Historical,EV stock,Cars,PHEV,2012,Vehicles,80.0


In [28]:
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
encoder.fit(df1[cat_cols])
df1[cat_cols]= encoder.transform(df1[cat_cols])
df1.head()
df1[df1.columns].nunique()

/tmp/ipykernel_6188/3614962638.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[cat_cols]= encoder.transform(df1[cat_cols])


region         36
category        3
parameter       2
mode            4
powertrain      2
year           15
unit            1
value         526
dtype: int64

In [29]:
encoder.fit(df2[cat_cols])
df2[cat_cols]= encoder.transform(df2[cat_cols])
df2.head()
df2[df2.columns].nunique()

/tmp/ipykernel_6188/704807145.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[cat_cols]= encoder.transform(df2[cat_cols])


region         36
category        3
parameter       2
mode            4
powertrain      1
year           15
unit            1
value         460
dtype: int64

In [30]:
encoder.fit(df3[cat_cols])
df3[cat_cols]= encoder.transform(df3[cat_cols])
df3.head()
df3[df3.columns].nunique()

/tmp/ipykernel_6188/3428284681.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3[cat_cols]= encoder.transform(df3[cat_cols])


region         36
category        2
parameter       1
mode            1
powertrain      2
year           15
unit            1
value         260
dtype: int64

In [31]:
encoder.fit(df4[cat_cols])
df4[cat_cols]= encoder.transform(df4[cat_cols])
df4.head()
df4[df4.columns].nunique()

/tmp/ipykernel_6188/3605974996.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4[cat_cols]= encoder.transform(df4[cat_cols])


region          6
category        3
parameter       1
mode            4
powertrain      1
year           15
unit            1
value         189
dtype: int64

In [32]:
encoder.fit(df5[cat_cols])
df5[cat_cols]= encoder.transform(df5[cat_cols])
df5.head()
df5[df5.columns].nunique()

/tmp/ipykernel_6188/1150526013.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5[cat_cols]= encoder.transform(df5[cat_cols])


region          6
category        3
parameter       1
mode            4
powertrain      1
year           15
unit            1
value         213
dtype: int64

In [33]:
encoder.fit(df6[cat_cols])
df6[cat_cols]= encoder.transform(df6[cat_cols])
df6.head()
df6[df6.columns].nunique()

/tmp/ipykernel_6188/124426687.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df6[cat_cols]= encoder.transform(df6[cat_cols])


region          6
category        3
parameter       1
mode            4
powertrain      1
year           15
unit            1
value         220
dtype: int64

In [34]:
for cat in df1.columns:
    IQR=df1[cat].quantile(0.75)-df1[cat].quantile(0.25)
    UB= df1[cat].quantile(0.75)+ (1.5*IQR)
    LB= df1[cat].quantile(0.25)-(1.5*IQR)
    s=0
    for row in df1[cat]:
        if row>=UB or row<= LB:
            s+=1
    print(f"{cat}: {s}")

region: 0
category: 4624
parameter: 0
mode: 0
powertrain: 0
year: 192
unit: 4624
value: 840


In [35]:
# for cat in df1.columns:
#     fig,ax = plt.subplots(nrows=1,ncols=2,figsize=(20,10))
#     sns.histplot(df1[cat],ax=ax[0],kde=True)
#     ax[0].set_title("Histplot")
#     sns.boxplot(df1[cat],ax=ax[1])
#     ax[1].set_title("Box_Plot")
#     plt.tight_layout()
#     plt.show()

In [36]:
from scipy.stats import shapiro
shapiro(df4)

ShapiroResult(statistic=0.09166896343231201, pvalue=0.0)

In [37]:
df1.head()

,region,category,parameter,mode,powertrain,year,unit,value
0,0.0,0.0,1.0,1.0,0.0,2011,0.0,49.0
3,0.0,0.0,0.0,1.0,0.0,2011,0.0,49.0
4,0.0,0.0,0.0,1.0,0.0,2012,0.0,170.0
7,0.0,0.0,1.0,1.0,0.0,2012,0.0,220.0
8,0.0,0.0,1.0,1.0,1.0,2012,0.0,80.0


In [38]:
from sklearn.model_selection import train_test_split
X=df1.iloc[:,:-1].values
Y=df1[['value']]
X_train , X_test, Y_train, Y_test =train_test_split(X,Y, test_size=0.2, random_state=32)

In [39]:
Y

,value
0,49.0
3,49.0
4,170.0
7,220.0
8,80.0
...,...
9537,910000.0
9538,12000000.0
9539,1300000.0
9540,180000000.0


In [40]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
# scaler = StandardScaler()
Y_train_scaled= scaler.fit_transform(Y_train)
Y_test_scaled = scaler.transform(Y_test)

In [42]:
X_train_scaled

array([[-1.5534551 , -0.44316373,  0.9688546 , ...,  1.29034321,
         0.36844904,  0.        ],
       [ 0.77472603, -0.44316373, -1.03214661, ..., -0.77498761,
        -0.52730712,  0.        ],
       [-1.5534551 , -0.44316373, -1.03214661, ..., -0.77498761,
        -0.07942904,  0.        ],
       ...,
       [-0.80843714, -0.44316373,  0.9688546 , ..., -0.77498761,
         0.36844904,  0.        ],
       [-0.71530989, -0.44316373, -1.03214661, ...,  1.29034321,
         0.14451   ,  0.        ],
       [ 0.3090898 , -0.44316373, -1.03214661, ..., -0.77498761,
         0.59238808,  0.        ]])

In [43]:
Y_train_scaled

array([[-0.10197648],
       [-0.1052677 ],
       [-0.10545325],
       ...,
       [-0.1052561 ],
       [-0.10406394],
       [-0.09061147]])

In [44]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
model = RandomForestRegressor()
model.fit(X_train_scaled, Y_train_scaled)
Y_pred_scaled= model.predict(X_test_scaled)
# Y_pred= scaler.inverse_transform(Y_pred_scaled)
eval = mean_squared_error(Y_test_scaled, Y_pred_scaled)


/home/ayush/Ayush/Sixth_Sem/ML/myenv/lib/python3.10/site-packages/sklearn/base.py:1351: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


ValueError: Expected 2D array, got 1D array instead:
array=[-1.05442720e-01 -1.01526873e-01 -9.98236267e-02 -1.05442175e-01
 -7.56467245e-02 -7.24167885e-02 -9.15352639e-02 -1.04978610e-01
  1.70054314e-02  1.74872150e+00 -9.59632343e-02 -1.37232137e-02
 -1.05303276e-01 -1.00127397e-01 -1.02321503e-01 -9.55786794e-02
 -1.05305904e-01 -1.05418825e-01 -1.05285818e-01 -1.05267351e-01
 -1.05453169e-01 -1.04329041e-01 -1.05405572e-01 -1.05291639e-01
 -1.03532447e-01 -1.05119568e-01 -1.02279211e-01  2.22960755e-01
 -1.05353771e-01 -1.05246446e-01 -1.04969541e-01 -9.54130518e-02
 -1.05439391e-01 -1.05378647e-01 -1.04740617e-01 -1.05417039e-01
 -3.90212858e-02 -1.05434087e-01 -1.03153180e-01 -1.05444158e-01
 -1.05381098e-01 -1.01076417e-01 -9.88469316e-02 -9.05850331e-02
 -1.05387091e-01 -1.04999893e-01 -1.01451606e-01 -1.05178237e-01
  1.09529255e+00 -1.05354597e-01 -1.04195895e-01 -1.04881590e-01
 -6.44586837e-02 -1.04749391e-01 -1.02421494e-01 -1.04064833e-01
 -3.12499382e-02 -1.05452800e-0